# Experimental data analysis and figure generation

This notebook analyzes experimental data obtained from expressing our select GFPs in *E. coli* and measuring their fluorescence using the SpectraMax iD3 plate reader. The notebook also creates figures for the pub.

## Setup and Imports

In [ ]:
from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import arcadia_pycolor as apc

# Set style for publication-quality figures
apc.mpl.setup()

In [ ]:
# Define data paths
OUTPUTS_DIR = Path("../experimental_data/")
FIGURES_DIR = Path("../figures/")
FIGURES_DIR.mkdir(exist_ok=True)

# Figure output paths
FIG1_PATH = FIGURES_DIR / 'training_data.svg'
FIG2_PATH = FIGURES_DIR / 'experimental_data.svg'

## Load data

In [ ]:
training_data = pd.read_csv("../data/seq_and_score.csv")
experimental_data_rep1_rep2 = pd.read_csv("../experimental_data/032125_rep1_rep2.csv", header=None)
experimental_data_rep3 = pd.read_csv("../experimental_data/042425_rep3.csv", header=None)
experimental_data_rep4 = pd.read_csv("../experimental_data/043025_rep4.csv", header=None)

## Figure 1: Comparison to training data

In [ ]:
# Normalize all data to wild-type for comparison
def add_wt_normalized_column(df):
    ref_score = df.loc[df['id'] == 'avgfp_00000', 'score']
    
    df['wt_normalized'] = df['score'] / ref_score.values[0]
    return df

normalized_sarkisyan_data = add_wt_normalized_column(training_data)

In [ ]:
sorted_data = normalized_sarkisyan_data.sort_values(by="wt_normalized", ascending=False).reset_index(drop=True)

# Bin the data to make graph size manageable
bin_size = 100
n_bins = len(sorted_data) // bin_size
binned_values = (
    sorted_data["wt_normalized"]
    .values[:n_bins * bin_size]
    .reshape(n_bins, bin_size)
    .mean(axis=1)
)

# Graph values
x = np.arange(len(binned_values))
fig, ax = plt.subplots(figsize=apc.mpl.get_figure_dimensions("float_square"))
bars = ax.bar(x, binned_values, zorder=1, width=1.0)

# Add gradient colors
cmap = apc.gradients.greens.to_mpl_cmap()
n = len(bars)
for i, bar in enumerate(bars):
    u = i / (n - 1)
    t = min(1.0, max(0.0, 0.5 + 0.5 * u**0.4)) # 0.4 for contrast
    bar.set_facecolor(cmap(t))

# Mark best, mid, worst values
targets = {"Best": 1.108, "Mid": 0.810, "Worst": 0.345}
for label, val in targets.items(): 
    idx = np.abs(binned_values - val).argmin()
    ax.axvline(x=idx, zorder=10, color=apc.black)
    ax.text(idx + 5, ax.get_ylim()[1] * 1, label, zorder=11)

apc.mpl.style_plot(ax, monospaced_axes="y")
plt.ylabel("Wild-type normalized score")
plt.xlabel("Sample")
ax.set_xticks([])
plt.show()
fig.savefig(FIG1_PATH, format="svg", bbox_inches="tight", dpi=300)

## Figure 2: Experimental data

### Parse plate reader data
We used a SpectraMax iD3 plate reader for our fluorescence intensity measures. We created a single file for each analysis and added a new plate for each measurement, naming the plate which day of the experiment we were on (day1, day2, etc.). We downloaded the raw XML file from the plate reader and turned it into a CSV file, which we parse here. Additionally, we're extracting day 2 data from our maturation data to use for our analysis as both the green and red fluorescence are reasonably bright and the cells are still healthy. We have four replicates of this data that can be found in the `experimental_data` directory.

The final output of this section is a dataframe (that's also saved as a CSV in the `experimental_data`) with the following columns: 
- sample: the sample name
- value_485_525: the fluorescence intensity at 485/525 (green) on day 2
- value_560_670: the fluorescence intensity at 560/670 (red) on day 2
- rep: the replicate number

In [ ]:
def extract_day2_data(df, well_map):
    plate_indices = df[df[0] == "Plate name"].index
    for idx in plate_indices:
        if str(df.iloc[idx, 1]).strip().lower() == "day2":
            day2_idx = idx
            break
    else:
        raise ValueError("No 'day2' plate found in file.")

    wl1_start = day2_idx + 42
    wl2_start = day2_idx + 52

    rows = list("ABCDEFGH")
    cols = list(range(1, 13))

    def extract_block(start_row):
        block = df.iloc[start_row:start_row + 8, 2:14].copy()
        block.index = rows
        block.columns = cols
        return block

    def melt_block(block, label):
        melted = block.stack().reset_index()
        melted.columns = ["row", "column", label]
        melted["well"] = melted["row"] + melted["column"].astype(str)
        return melted[["well", label]]

    block_485 = extract_block(wl1_start)
    block_670 = extract_block(wl2_start)

    melted_485 = melt_block(block_485, "value_485_525")
    melted_670 = melt_block(block_670, "value_560_670")

    merged = pd.merge(melted_485, melted_670, on="well")
    merged["sample"] = merged["well"].map(well_map)
    merged = merged.dropna(subset=["sample"])

    return merged[["sample", "value_485_525", "value_560_670"]]

well_map = {
    "A1": "GFP1_1", 
    "A2": "GFP1_2", 
    "A3": "GFP1_3",
    "A4": "GFP1_4", 
    "A5": "GFP1_5",
    "A6": "GFP1_6",
    "A7": "GFP1_7",
    "A8": "GFP1_8",
    "A9": "GFP1_9",
    "A10": "GFP1_10",
    "B1": "Wild-type",
    "B2": "Best", 
    "B3": "Mid", 
    "B4": "Worst",
    "B6": "Empty_vector"
}

well_map_rep2 = {
    "C1": "GFP1_1", 
    "C2": "GFP1_2", 
    "C3": "GFP1_3",
    "C4": "GFP1_4", 
    "C5": "GFP1_5",
    "C6": "GFP1_6",
    "C7": "GFP1_7",
    "C8": "GFP1_8",
    "C9": "GFP1_9",
    "C10": "GFP1_10",
    "D1": "Wild-type",
    "D2": "Best", 
    "D3": "Mid", 
    "D4": "Worst", 
    "D6": "Empty_vector"
}

exp_data_rep1 = extract_day2_data(experimental_data_rep1_rep2, well_map)
exp_data_rep2 = extract_day2_data(experimental_data_rep1_rep2, well_map_rep2)
exp_data_rep3 = extract_day2_data(experimental_data_rep3, well_map)
exp_data_rep4 = extract_day2_data(experimental_data_rep4, well_map)

exp_data_rep1["rep"] = 1
exp_data_rep2["rep"] = 2
exp_data_rep3["rep"] = 3
exp_data_rep4["rep"] = 4

compiled_experimental_data = pd.concat(
    [exp_data_rep1, exp_data_rep2, exp_data_rep3, exp_data_rep4],
    ignore_index=True
)

compiled_experimental_data.to_csv("../experimental_data/compiled_experimental_data.csv", index=False)

### Background subtraction and normalization
In addition to measuring the fluorescence values for our samples of interest, we also measured the values for cells containing an empty vector with no fluorescent protein to use as a background control. Here, we subtract the green and red fluorescent values for the empty_vector from the corresponding values for the sample. 

Additionally, we created a fusion protein where RFP serves as an expression control for our GFP. Therefore, we normalize our GFP fluorescence, by dividing by the RFP fluorescence. 

In [ ]:
# Subtract background
def subtract_background(df: pd.DataFrame, empty_label: str = "Empty_vector", suffix: str = "_bkgd_sub",) -> pd.DataFrame:
    CHANNELS = ["value_485_525", "value_560_670"]
    df_out = df.copy()
    empty = df_out[df_out["sample"] == empty_label]
    keys = ["rep"]
    bg = empty.set_index(keys)[CHANNELS]
    merged = df_out.merge(bg, left_on=keys, right_index=True, how="left", suffixes=("", "_bkgd"))
    for col in CHANNELS:
        merged[f"{col}{suffix}"] = merged[col] - merged[f"{col}_bkgd"]
    return merged

# Normalize GFP data using RFP data
def compute_green_red_ratio(df):
    df = df.copy()
    df['Green/Red'] = df['value_485_525_bkgd_sub'] / df['value_560_670_bkgd_sub']
    return df

experimental_data_day_2_bkgd_subtract = subtract_background(compiled_experimental_data)
normalized_experimental_data = compute_green_red_ratio(experimental_data_day_2_bkgd_subtract)

### Normalizing to wild-type
In order to qualitatively compare our data to the data from Sarkisyan et al., we normalized to the wild-type GFP/RFP ratio. 

In [ ]:
def normalize_to_wt(df, wt_label="Wild-type", value_col="Green/Red", suffix="_wt_normalized"):
    df = df.copy()
    
    # Filter WT values for each replicate
    wt_values = df[df["sample"] == wt_label][["rep", value_col]].rename(columns={value_col: "wt_value"})

    # Merge WT values back onto all rows by replicate
    df = df.merge(wt_values, on="rep", how="left")

    # Normalize
    df[f"{value_col}{suffix}"] = df[value_col] / df["wt_value"]

    return df

def summarize_replicates(df):
    summary = (
        df.groupby(["sample"])
          .agg({
              "Green/Red_wt_normalized": ["mean", "std"]
          })
    )
    summary.columns = ["_".join(col).strip() for col in summary.columns.values]
    summary = summary.reset_index()
    return summary


normalized_to_wt_exp_data = normalize_to_wt(normalized_experimental_data)
averaged_wt_normalized_data = summarize_replicates(normalized_to_wt_exp_data)

### Graphing

In [ ]:
# Graph normalized data
sorted_data = averaged_wt_normalized_data.sort_values(by='Green/Red_wt_normalized_mean', ascending=False).reset_index(drop=True)

# Create bar plot
fig, ax = plt.subplots(figsize=apc.mpl.get_figure_dimensions("half_square"), layout="constrained")
bars = plt.bar(
    sorted_data["sample"],
    sorted_data["Green/Red_wt_normalized_mean"],
    edgecolor=apc.black,
    linewidth=0.5,
    zorder=1,
)

# Add gradient colors
cmap = apc.gradients.greens.to_mpl_cmap()
n = len(bars)
for i, bar in enumerate(bars):
    u = i / (n - 1) if n > 1 else 0.5
    t = min(1.0, max(0.0, 0.5 + 0.5 * u**0.4))  # contrast = 0.4
    bar.set_facecolor(cmap(t))

# Re-color controls
override_colors = {
    "Wild-type": apc.mud,
    "Best": apc.mud,
    "Mid": apc.mud,
    "Worst": apc.mud,
    "Empty_vector": apc.mud,
}
for bar, name in zip(bars, sorted_data["sample"]):
    if name in override_colors:
        bar.set_facecolor(override_colors[name])

# Add scatter plot
rep_offset = {1: -0.1, 2: -0.07, 3: 0.07, 4: 0.1} 
for i, name in enumerate(sorted_data["sample"]):
    grp = normalized_to_wt_exp_data.loc[
        normalized_to_wt_exp_data["sample"] == name, ["Green/Red_wt_normalized", "rep"]
    ].dropna()

    if grp.empty:
        continue

    ys = grp["Green/Red_wt_normalized"].to_numpy(dtype=float)
    reps = grp["rep"].to_numpy()
    m = np.isfinite(ys)
    ys, reps = ys[m], reps[m]
    if ys.size == 0:
        continue

    xs = i + np.array([rep_offset.get(int(r), 0.0) for r in reps])
    ax.scatter(xs, ys, s=28, facecolor=apc.black, zorder=3)

apc.mpl.style_plot(ax, monospaced_axes="y")
plt.ylabel("GFP/RFP ratio (AU)")
plt.xlabel("Sample")
plt.xticks(rotation=45, ha="right")
plt.show()
fig.savefig(FIG2_PATH, format="svg", bbox_inches="tight", dpi=300)